## Step 1)

Import the necessary packages, enter your api key, and read the input file. 


In [6]:
import folium
import numpy as np
import pandas as pd
from geopy.geocoders import MapBox
from ipywidgets import FloatProgress
from IPython.display import display

# You need this to query mapbox
api_key="pk.eyJ1IjoianVuZXNwYWNlYm9vdHMiLCJhIjoiY2twY3g4aXloMWFlcDJzbXN3aG95aG5uZiJ9.mFiJt0MIfL1MiJ2rB2xhKQ"

# Sepcifiy the location of your data
input_filename='TestData1.csv'
output_filename = 'Output.csv'


if api_key == "":
    print('Please enter your API key to continue!')
else:
    Data = pd.read_csv('Input/'+input_filename)
Data.head()

,Address,City/Location,State/Province,Postal Code,Country,Name
0,NaN,Jasper National Park,Alberta,NaN,Canada,Test point 1
1,NaN,Tuktoyaktuk,Northwest Territories,NaN,Canada,Test point 2
2,1984 West Mall,University of British Columbia,British Columbia,V6T 1Z2,Canada,Test point 3
3,6 Mud Bay Rd,Haines,Alaska,99827,United States,Test point 4


## Step 2)

Specify the geocode request format

In [7]:
location_columns = ['Address','City/Location','State/Province','Postal Code','Country']
## Use this to add exra info that isn't specified (eg. missing country/province)
additional_info = ''#, Canada'

attempt = ', '.join(x for x in location_columns)+ additional_info

print('attempt format')
print(attempt)

attempt format
Address, City/Location, State/Province, Postal Code, Country


## Step 3)

Geocode the points

In [8]:
Data['latitude'] = np.nan
Data['longitude'] = np.nan
Data['geocoding_Notes'] = ''

prog = FloatProgress(min=0, max=100,description='Progress:')
prog.value=0
display(prog)


geolocator = MapBox(api_key=api_key)
for index, row in Data.iterrows():
    row[location_columns]=row[location_columns].fillna('')
    attempt = ' '.join(row[x] for x in location_columns) +' ' + additional_info
    prog.value=(index+1)/len(Data)*100
    try:
        g = geolocator.geocode(attempt,timeout=3)
        Data.loc[Data.index==index,['latitude','longitude']]=g.latitude,g.longitude
    except:
        Data.loc[Data.index==index,'geocoding_Notes']='Geocoding Failed'
        pass
Data.to_csv('Output/'+output_filename)
Failures = Data.loc[Data['geocoding_Notes']=='Geocoding Failed']

print('Geocoding Done')
print()
if Failures.count()[location_columns[0]]>0:
    print('Filed to Geocode: ',Failures.shape[0], ' records.')
    print(Failures)
else:
    print('All Records Geocoded')

FloatProgress(value=0.0, description='Progress:')

 Jasper National Park Alberta  Canada 
 Tuktoyaktuk Northwest Territories  Canada 
1984 West Mall University of British Columbia British Columbia V6T 1Z2 Canada 
6 Mud Bay Rd Haines Alaska 99827 United States 
Geocoding Done

All Records Geocoded


## Step 4)

Inspect the results.  Create a web map to display the points.

In [9]:
# your mapbox api key also gives you access to the mapbox satelite basemap
token = api_key
tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

Map = folium.Map(location=[53, -125],zoom_start=3, tiles=tileurl, attr='Mapbox')

for index, row in Data.loc[Data['geocoding_Notes']!='Geocoding Failed'].iterrows():
    row[location_columns]=row[location_columns].fillna('')
    attempt = ' '.join(row[x] for x in location_columns) +' ' + additional_info
    folium.CircleMarker(location=[row['latitude'],row['longitude']],
                        popup=attempt,
                        fill_color='blue',
                        color='black'
                       ).add_to(Map)
Map